In [9]:
import sys
sys.path.append("../../src/")
import numpy as np
import math
import uncertainpy.gradual as grad

In [10]:
from itertools import combinations

# generate random combination with number n
def combine(temp_list, n):
    return list(combinations(temp_list, n))

In [11]:
# set semantics and compute strength of arguments
bag = grad.BAG("../../bags/fig3.bag")

# DF-QuAD semantics
agg_f = grad.semantics.modular.ProductAggregation()
inf_f = grad.semantics.modular.LinearInfluence(conservativeness=1)

# QE semantics
# agg_f = grad.semantics.modular.SumAggregation()
# inf_f = grad.semantics.modular.QuadraticMaximumInfluence(conservativeness=1)

strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

In [12]:
# get all the attacks and supports in QBAF
att_relation = bag.get_attacks()
sup_relation = bag.get_supports()
all_relation = att_relation + sup_relation

In [13]:
# rename relation concisely
all_relation_new = []
for i in all_relation:
    if str(i)[0:3]=="Att":
        all_relation_new.append(("att",str(i.get_attacker().get_name()),str(i.get_attacked().get_name())))
    elif str(i)[0:3]=="Sup":
        all_relation_new.append(("sup",str(i.get_supporter().get_name()),str(i.get_supported().get_name())))
all_relation_new

[('att', 'delta', 'beta'),
 ('att', 'delta', 'gamma'),
 ('att', 'zeta', 'delta'),
 ('sup', 'beta', 'alpha'),
 ('sup', 'gamma', 'alpha')]

In [14]:
# set topic argument
topic_a = "alpha"
topic_arg = bag.arguments[topic_a]

In [15]:
# compute strength for all arguments
for arg in bag.arguments.values():
    print((arg.name,arg.initial_weight,arg.strength))

('alpha', 0.5, 0.8046875)
('beta', 0.5, 0.375)
('gamma', 0.5, 0.375)
('delta', 0.5, 0.25)
('zeta', 0.5, 0.5)


In [16]:
n = len(all_relation_new) # the number of all relations in the AF
attribution = {} # store the attribution of all the relations

for i in range(len(all_relation_new)):

    # split current and the rest relation
    cur_relation = all_relation_new[i]
    rest_relation = all_relation_new.copy()
    rest_relation.remove(cur_relation)

    # compute combinations for the rest_relation
    combine_rest_relation = []
    for j in range(len(rest_relation)+1):
        combine_rest_relation.extend(combine(rest_relation, j))

    difference = []
    coefficient = []

    # remove relations in combine_rest_relation
    for j in combine_rest_relation:
        for k in j:
            # remove att
            if k[0]=="att":
                temp_attacker = bag.remove_attack(k[1],k[2]) # k[1] attacks k[2]
            # remove sup
            elif k[0]=="sup":
                temp_supporter = bag.remove_support(k[1],k[2]) # k[1] supports k[2]

        # compute sigma(S U {i})
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        topic_arg = bag.arguments[topic_a]
        strength_before_drop_cur = topic_arg.strength

        # remove cur_relation
        if cur_relation[0]=="att":
            temp_attacker = bag.remove_attack(cur_relation[1],cur_relation[2])
        elif cur_relation[0]=="sup":
            temp_supporter = bag.remove_support(cur_relation[1],cur_relation[2])

        # compute sigma(S)
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)
        topic_arg = bag.arguments[topic_a]
        strength_after_drop_cur = topic_arg.strength

        # compute sigma(S U {i})-sigma(S)
        difference.append(strength_before_drop_cur-strength_after_drop_cur)

        # compute the coefficient
        S = len(j)
        coefficient.append(math.factorial(n-1-S) * math.factorial(S) / math.factorial(n))

        # recover QBAF for the next iteration
        bag = grad.BAG("../../bags/fig3.bag")
        strength_values = grad.algorithms.computeStrengthValues(bag, agg_f, inf_f)

    # compute attributions
    attribution[cur_relation] = sum(np.multiply(coefficient, difference))

[(), (('att', 'delta', 'gamma'),), (('att', 'zeta', 'delta'),), (('sup', 'beta', 'alpha'),), (('sup', 'gamma', 'alpha'),), (('att', 'delta', 'gamma'), ('att', 'zeta', 'delta')), (('att', 'delta', 'gamma'), ('sup', 'beta', 'alpha')), (('att', 'delta', 'gamma'), ('sup', 'gamma', 'alpha')), (('att', 'zeta', 'delta'), ('sup', 'beta', 'alpha')), (('att', 'zeta', 'delta'), ('sup', 'gamma', 'alpha')), (('sup', 'beta', 'alpha'), ('sup', 'gamma', 'alpha')), (('att', 'delta', 'gamma'), ('att', 'zeta', 'delta'), ('sup', 'beta', 'alpha')), (('att', 'delta', 'gamma'), ('att', 'zeta', 'delta'), ('sup', 'gamma', 'alpha')), (('att', 'delta', 'gamma'), ('sup', 'beta', 'alpha'), ('sup', 'gamma', 'alpha')), (('att', 'zeta', 'delta'), ('sup', 'beta', 'alpha'), ('sup', 'gamma', 'alpha')), (('att', 'delta', 'gamma'), ('att', 'zeta', 'delta'), ('sup', 'beta', 'alpha'), ('sup', 'gamma', 'alpha'))]


NameError: name 'ssss' is not defined

In [ ]:
attribution